In [ ]:
import io
import numpy as np
import pandas as pd
from copy import deepcopy
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
uploaded = files.upload()

Saving njr_data.csv to njr_data (1).csv


In [ ]:
data_df = pd.read_csv(io.BytesIO(uploaded["njr_data.csv"]))
data_df

,Unnamed: 0,player,position,games,goals,shots,time,xG,assists,xA,key_passes,season,team,yellow,red,npg,npxG,xGChain,xGBuildup
0,0,Neymar,FWL,10,3,20,840,3.67,3,2.87,23,2021,Paris Saint Germain,2,0,2,2.91,9.96,5.55
1,1,Neymar,FWL,18,9,70,1413,13.25,5,7.52,58,2020,Paris Saint Germain,5,2,4,8.69,21.54,9.85
2,2,Neymar,AMC,15,12,70,1325,14.73,6,5.25,38,2019,Paris Saint Germain,2,1,8,10.92,21.30,7.21
3,3,Neymar,FW,17,15,55,1443,11.90,7,7.78,43,2018,Paris Saint Germain,2,0,10,8.10,18.17,6.62
4,4,Neymar,FWL,20,19,91,1797,16.54,13,12.28,70,2017,Paris Saint Germain,4,1,15,13.50,31.43,11.52
5,5,Neymar,FWL,30,13,105,2652,14.18,11,10.67,91,2016,Barcelona,6,1,12,13.43,30.41,12.35
6,6,Neymar,FWL,34,24,124,3058,28.05,12,18.07,102,2015,Barcelona,6,0,19,22.85,52.41,22.22
7,7,Neymar,Sub,33,22,95,2573,22.72,7,8.26,52,2014,Barcelona,6,0,21,21.98,36.89,11.68


In [ ]:
data_df[data_df["team"].str.contains("Barcelona")]

,Unnamed: 0,player,position,games,goals,shots,time,xG,assists,xA,key_passes,season,team,yellow,red,npg,npxG,xGChain,xGBuildup
5,5,Neymar,FWL,30,13,105,2652,14.18,11,10.67,91,2016,Barcelona,6,1,12,13.43,30.41,12.35
6,6,Neymar,FWL,34,24,124,3058,28.05,12,18.07,102,2015,Barcelona,6,0,19,22.85,52.41,22.22
7,7,Neymar,Sub,33,22,95,2573,22.72,7,8.26,52,2014,Barcelona,6,0,21,21.98,36.89,11.68


In [ ]:
mins_df = data_df.groupby(["team"])["time"].sum().reset_index()
px.bar(x = "team",y = "time" , text = "time", data_frame = mins_df)

In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8 non-null      int64  
 1   player          8 non-null      object 
 2   position        8 non-null      object 
 3   games           8 non-null      int64  
 4   goals           8 non-null      int64  
 5   shots           8 non-null      int64  
 6   time            8 non-null      int64  
 7   xG              8 non-null      float64
 8   assists         8 non-null      int64  
 9   xA              8 non-null      float64
 10  key_passes      8 non-null      int64  
 11  season          8 non-null      int64  
 12  team            8 non-null      object 
 13  yellow          8 non-null      int64  
 14  red             8 non-null      int64  
 15  npg             8 non-null      int64  
 16  npxG            8 non-null      float64
 17  xGChain         8 non-null      float64

In [ ]:
data_df["goalsPer90"] = data_df["goals"].divide(data_df["time"]).multiply(90)
data_df["assistsPer90"] = data_df["assists"].divide(data_df["time"]).multiply(90)
per90Cols = ["goals", "assists", "xGChain",  "xGBuildup",
             "npxG", "npg", 'xG',"xA"]
for col in per90Cols:
    data_df[col + "Per90"] = data_df[col].divide(data_df["time"]).multiply(90)
cols_for_radar = [i + "Per90" for i in per90Cols]
print(cols_for_radar)

['goalsPer90', 'assistsPer90', 'xGChainPer90', 'xGBuildupPer90', 'npxGPer90', 'npgPer90', 'xGPer90', 'xAPer90']


In [ ]:
data_df.loc[(data_df["team"] == "Paris Saint Germain"), cols_for_radar].sum()

goalsPer90        3.596908
assistsPer90      2.135123
xGChainPer90      6.593297
xGBuildupPer90    2.701619
npxGPer90         2.788349
npgPer90          2.387412
xGPer90           3.808276
xAPer90           2.243349
dtype: float64

In [ ]:
data_df.loc[(data_df["team"] == "Barcelona"), cols_for_radar].sum()

goalsPer90        1.917050
assistsPer90      0.971326
xGChainPer90      3.864854
xGBuildupPer90    1.481625
npxGPer90         1.897098
npgPer90          1.700980
xGPer90           2.101476
xAPer90           1.182846
dtype: float64

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == "Barcelona"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == "Paris Saint Germain"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_df[cols_for_radar] = scaler.fit_transform(data_df[cols_for_radar])

In [ ]:
fig = px.line_polar(data_df, r=data_df.loc[(data_df["player"] == "Neymar"), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
for i, row in data_df.iterrows():
    print(row["team"])
    fig = px.line_polar(data_df, r=data_df.loc[(data_df["team"] == row["team"]), cols_for_radar].sum(),
                        theta=cols_for_radar, line_close=True,
                        title=row["team"])
    fig.update_traces(fill='toself')
    fig.show()

Paris Saint Germain


Paris Saint Germain


Paris Saint Germain


Paris Saint Germain


Paris Saint Germain


Barcelona


Barcelona


Barcelona


In [ ]:
id_names = data_df.drop_duplicates(subset=["id"])[["id", "player_name"]]
id_names.shape
id_names.head()
df_max = data_df[cols_for_radar].max().max()

In [ ]:
import plotly.graph_objects as go
for i, row in id_names.iterrows():
    if row["id"] == 647:
        continue
    print(row["player_name"])
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=data_df.loc[(data_df["id"] == 647), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name='Harry Kane'))
    fig.add_trace(go.Scatterpolar(
        r=data_df.loc[(data_df["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["player_name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title="Harry Kane vs " + row["player_name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, df_max]
        )),
    showlegend=True
    )
    fig.show()

Robert Lewandowski


André Silva


Erling Haaland


Andrej Kramaric


Wout Weghorst


...

Simy


Lorenzo Insigne


Domenico Berardi


Lautaro Martínez


João Pedro
